# 2. Tensorflow Prophet Testing

## I - Setup

In [1]:
# Import of librairies
import tensorflow as tf
import mysql.connector as mariadb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tqdm import tqdm
from joblib import load

class sql_query:
    def __init__(self, credentials_path):
        self.db_credentials = pd.read_csv(credentials_path, index_col="Field")
      
    
    def __call__(self, query):
        
        mariadb_connection = mariadb.connect(
            user=self.db_credentials.loc["user"][0],
            password=self.db_credentials.loc["password"][0],
            host=self.db_credentials.loc["host"][0],
            port=3306,
            db = "db_velib")
        
        self.cursor = mariadb_connection.cursor()
        cursor = self.cursor
        cursor.execute("SET  time_zone = 'Europe/Paris'")
        cursor.execute(query)
        field_names = [i[0] for i in cursor.description]
        df = pd.DataFrame(cursor, columns=field_names)
        return df
    
# Transforming the input data in the proper format 


def measure_rmse(actual, predicted):
    return math.sqrt(mean_squared_error(actual, predicted))


In [3]:
# Extracting base for prediction 

request = sql_query("../../aws_mariadb_crendentials.csv")

query = """
SELECT station_id, date_of_update, nb_total_free_bikes FROM db_velib.velib_realtime
WHERE date_of_update >= DATE_SUB(NOW(), INTERVAL 185 Minute) AND MINUTE(date_of_update)%5 = 0
ORDER BY station_id, date_of_update ASC;
"""
df= request(query)
df.index = df.date_of_update
df = df[['station_id','nb_total_free_bikes']]
df = df.pivot_table(df, index= 'station_id', columns=df.index)

# Creating dataframe for proper predction
                    
df_prediction = pd.DataFrame(index=df.index, columns=['last_observations','Model_A', 'Model_B'])
                    
for i in df_prediction.index:
    df_prediction["last_observations"].loc[i] = np.array(df.loc[i])

In [4]:
# Extracting the list of the stations

request = sql_query("../../aws_mariadb_crendentials.csv")
query = """
SELECT DISTINCT station_id FROM velib_realtime
"""
df= request(query)
# Removing bad values
df= df.drop(0)
df = df.drop(1391)
list_of_stations = list(df.station_id)
print(list_of_stations[0:5])

[1001, 1002, 1003, 1006, 1007]


In [5]:
# limiting for testing purpuse
list_of_stations= list_of_stations[0:4]
print(list_of_stations)

[1001, 1002, 1003, 1006]


In [6]:
# Main pipelinhe

# Variables
day_of_testing = '2020-05-17'
past_history = 36
future_target = 6

# Request for database

for station_id in tqdm(list_of_stations):
    
    # Importing models

    LSTM_model_A = tf.keras.models.load_model('/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - LSTM_A.h5'.format(day_of_testing, station_id))
    LSTM_model_B = tf.keras.models.load_model('/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - LSTM_B.h5'.format(day_of_testing, station_id))
    std = load('/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate - {} - {} - std.joblib'.format(day_of_testing, station_id)) 
    
    
    # Std Scaling
    
    input_data = std.transform(df_prediction[df_prediction.index == station_id]["last_observations"].iloc[0].reshape(-1, 1))[-36:]
    # Exporting results
    
    df_prediction.loc[station_id]['Model_A'] = std.inverse_transform(LSTM_model_A.predict(input_data.reshape(1,past_history,1))[0])
    df_prediction.loc[station_id]['Model_B'] = std.inverse_transform(LSTM_model_B.predict(input_data.reshape(1,past_history,1))[0])

    
   # df_results.to_csv("/home/exalis/Github/velib-prediction-v2/4. Models/Tensorflow Univariate Results - {} - {}.csv".format(day_of_testing, station_id))
    
    
    print('finished ', station_id)

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:01<00:04,  1.56s/it]

finished  1001


 50%|█████     | 2/4 [00:02<00:02,  1.36s/it]

finished  1002


 75%|███████▌  | 3/4 [00:03<00:01,  1.22s/it]

finished  1003


100%|██████████| 4/4 [00:04<00:00,  1.09s/it]

finished  1006
